In [1]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
import os

# Specify the path to the existing database file
db_path = r"..-Relational Database Design\DEL 5 - 2\D5\Final - 02\womenwordcup2023_module5.db"

# Check if the database file exists
if not os.path.exists(db_path):
    messagebox.showerror("Error", f"Database file {db_path} does not exist. Please check the path.")
    exit(1)

# Connect to the SQLite database
conn = sqlite3.connect(db_path)
c = conn.cursor()

# GUI Application
class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Database Manager - Program 3")

        self.create_widgets()
    def create_widgets(self):
        # Frame for buttons
        btn_frame = tk.Frame(self.root)
        btn_frame.pack(pady=10)

        self.query1_btn = tk.Button(btn_frame, text="Rank Rounds by Goal Count", command=self.rank_rounds_by_goal_count)
        self.query1_btn.grid(row=0, column=0, padx=5)

        self.query2_btn = tk.Button(btn_frame, text="Rank Players by Goal Count", command=self.rank_players_by_goal_count)
        self.query2_btn.grid(row=1, column=0, padx=5)

        self.query3_btn = tk.Button(btn_frame, text="Rank Teams by Wins", command=self.rank_teams_by_wins)
        self.query3_btn.grid(row=3, column=0, padx=5)

        self.query4_btn = tk.Button(btn_frame, text="Cumulative Distribution of Wins", command=self.cumulative_distribution_of_wins)
        self.query4_btn.grid(row=4, column=0, padx=5)

        self.query5_btn = tk.Button(btn_frame, text="Running Total Goals by Player", command=self.running_total_goals_by_player)
        self.query5_btn.grid(row=5, column=0, padx=5)

        self.query6_btn = tk.Button(btn_frame, text="Detailed Match Information", command=self.detailed_match_information)
        self.query6_btn.grid(row=6, column=0, padx=5)

        self.query7_btn = tk.Button(btn_frame, text="Manager with Higher Than Average Wins", command=self.manager_with_higher_Than_Average_wins)
        self.query7_btn.grid(row=7, column=0, padx=5)

        self.query8_btn = tk.Button(btn_frame, text="Manager with Higher Than Average Goals", command=self.manager_with_higher_Than_Average_goals)
        self.query8_btn.grid(row=8, column=0, padx=5)


        # Frame for output
        self.output_frame = tk.Frame(self.root)
        self.output_frame.pack(pady=10)

    def run_query(self, query, column_titles):
        self.clear_output_frame()
        try:
            c.execute(query)
            records = c.fetchall()
            # Display column titles
            title_row = tk.Frame(self.output_frame)
            title_row.pack()
            for title in column_titles:
                title_label = tk.Label(title_row, text=title, borderwidth=1, relief="solid", width=20)
                title_label.pack(side="left")
            # Display query results
            for record in records:
                data_row = tk.Frame(self.output_frame)
                data_row.pack()
                for item in record:
                    data_label = tk.Label(data_row, text=item, borderwidth=1, relief="solid", width=20)
                    data_label.pack(side="left")
        except sqlite3.Error as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def clear_output_frame(self):
        for widget in self.output_frame.winfo_children():
            widget.destroy()

    def rank_rounds_by_goal_count(self):
        self.run_query("""
            SELECT
                RANK() OVER (
                    ORDER BY
                        SUM(s.Goal_Count) DESC
                ) AS Round_Rank,
                r.Round_Name,
                SUM(s.Goal_Count) AS Total_Goal_Count                
            FROM Score s
                JOIN Round r ON r.RoundID = s.RoundID
            GROUP BY
                r.RoundID,
                r.Round_Name
            ORDER BY Round_Rank;
        """, ("Round Rank","Round Name", "Total Goal Count"))

    def rank_players_by_goal_count(self):
        self.run_query("""
            SELECT
                RANK() OVER (
                ORDER BY
                    SUM(s.Goal_Count) DESC
                ) AS Player_Rank,
                DENSE_RANK() OVER (
                    ORDER BY
                        SUM(s.Goal_Count) DESC
                ) AS Player_Dense_Rank,                
                p.Player_First_Name,
                p.Player_Last_Name,
                SUM(s.Goal_Count) AS Total_Goal_Count
            
            FROM Score s
                JOIN Player p ON p.PlayerID = s.PlayerID
            GROUP BY
                p.PlayerID, 
                p.Player_First_Name, 
                p.Player_Last_Name
            ORDER BY Player_Rank;
        """, ("Player Rank", "Dense Rank","First Name", "Last Name", "Total Goal Count"))

    
    def rank_teams_by_wins(self):
        self.run_query("""
            SELECT
                RANK () OVER (
                    ORDER BY
                        Win DESC
                ) AS W_Rank,
                Team_Name,
                Win,
                
                ROUND (
                    PERCENT_RANK() OVER (
                        ORDER BY
                            Win
                    ),
                    2
                ) AS W_Percen_Rank,
                ROUND (
                PERCENT_RANK() OVER (
                    ORDER BY
                        Win
                ),
                2
            ) *100 AS 'W_Percen_Rank (%age)'
            FROM Team
            Order BY W_Rank;
        """, ("Team Rank", "Team Name", "Wins", "Percent Rank", "Percent Rank (%)"))

    def cumulative_distribution_of_wins(self):
        self.run_query("""
            SELECT
                Team_Name,
                Win,
                ROUND(
                    CUME_DIST() OVER (
                        ORDER BY
                            Win
                    ),
                    2
                ) AS W_Cume_Dist,
                ROUND(
                    ROUND(
                        CUME_DIST() OVER (
                            ORDER BY
                                Win
                        ),
                        2
                    ) * 100,
                    2
                ) AS W_Cume_Dist_Percentage
            FROM Team;
        """, ("Team Name", "Wins", "Cumulative Distribution", "Cumulative Distribution (%)"))

    def running_total_goals_by_player(self):
        self.run_query("""
            SELECT
                p.Player_First_Name,
                p.Player_Last_Name,
                s.MatchID,
                g.Match_Date,
                s.Goal_Count,
                SUM(s.Goal_Count) OVER (
                    PARTITION BY s.PlayerID
                    ORDER BY
                        g.Match_Date ROWS BETWEEN UNBOUNDED PRECEDING
                        AND CURRENT ROW
                ) AS Running_Total_Goals
            FROM Score s
                JOIN Game g ON s.MatchID = g.MatchID
                JOIN Player p ON s.PlayerID = p.PlayerID
            ORDER BY
                p.Player_First_Name,
                g.Match_Date;
        """, ("First Name", "Last Name","MatchID", "Match Date", "Goal Count", "Running Total Goals"))
    def detailed_match_information(self):
        self.run_query("""
            SELECT
                g.MatchID,
                g.Match_Date, 
                t.Team_Name, 
                m.Manager_First_Name || ' ' || m.Manager_Last_Name AS Manager_Name,
                g.Attendance,
                l.Location_Name,
                l.City,
                g.Result, 
                r.Referee_First_Name || ' ' || r.Referee_Last_Name AS Referee_Name
            FROM Score s
                JOIN Game g ON g.MatchID = s.MatchID
                JOIN Referee r ON s.RefereeID = r.RefereeID
                JOIN Location l ON s.LocationID = l.LocationID
                JOIN Manager m ON s.ManagerID = m.ManagerID
                JOIN Team t ON s.TeamID = t.TeamID 
            ORDER BY g.MatchID;
        """, ("MatchID", "Match Date", "Team Name", "Manager Name", "Attendance", "Location Name", "City", "Result", "Referee Name"))

 
    def manager_with_higher_Than_Average_wins(self):
        self.run_query("""
            SELECT DISTINCT
                m.Manager_First_Name,
                m.Manager_Last_Name,
                t.Team_Name,
                t.Win AS Total_Wins
            FROM Score s
            JOIN Team t ON s.TeamID = t.TeamID
            JOIN Manager m ON s.ManagerID = m.ManagerID
            WHERE t.Win > (
                SELECT AVG(Win)
                FROM Team
                )
            Order BY Total_Wins DESC;
        """, ("First Name", "Last Name", "Team Name", "Total Wins"))
        
    def manager_with_higher_Than_Average_goals(self):
            self.run_query("""
                SELECT
                    m.Manager_First_Name,
                    m.Manager_Last_Name,
                    t.Team_Name,
                    SUM(s.Goal_Count) AS Total_Goals
                FROM Score s
                JOIN Team t ON s.TeamID = t.TeamID
                JOIN Manager m ON m.ManagerID = s.ManagerID
                GROUP BY
                   m.ManagerID,
                   m.Manager_First_Name,
                   m.Manager_Last_Name,
                   t.Team_Name
                HAVING Total_Goals > (
                    SELECT AVG(Total_Goals)
                    FROM (
                        SELECT
                            SUM(s.Goal_Count) AS Total_Goals
                        FROM Score s
                        GROUP BY s.ManagerID
                   )
              )
              Order BY Total_Goals DESC;              
            """, ("First Name", "Last Name", "Team Name", "Total Goals"))





def main():
    root = tk.Tk()
    app = App(root)
    root.mainloop()

if __name__ == "__main__":
    main()

# Close the connection when the application exits
conn.close()
